In [1]:
# Input 

In [3]:
import sys
import os

# Füge /app zum Suchpfad hinzu
sys.path.append(os.path.abspath("../app"))

from input_manager import load_resume_data, extract_clean_text_from_pdf, find_csv_and_pdf_files
from rag_manager import init_chroma_db, get_collection, add_dataframe_to_chroma

In [4]:
csv_path, pdf_path = find_csv_and_pdf_files('../data/input')
df = load_resume_data(csv_path[0])
pdf_text = extract_clean_text_from_pdf(pdf_path[0])

📄 Gefundene CSV-Dateien: 1
📄 Gefundene PDF-Dateien: 1
✅ Es wurden 34 Berufserfahrungen erkannt. Der Zeitraum erstreckt sich von January 2010 bis June 2025 (5655 Tage).
✅ Qualitätsprüfung bestanden.
✅ Die Stellenauschreibung wurde erkannt.


In [5]:
# Init DB
db = init_chroma_db()
collection = get_collection(db, name="bewerbung")

# CSV laden
df = load_resume_data("data/input/resume_data.csv")

# In RAG speichern
add_dataframe_to_chroma(df, collection, source_id="resume_2025_06")

📂 Chroma-Verzeichnis gefunden. Vorhandene Daten werden geladen.


ValueError: The CHROMA_OPENAI_API_KEY environment variable is not set.

In [4]:
# Kurzprofil erstellen 

In [25]:
import tiktoken
import pandas as pd
from openai import OpenAI
import time

In [6]:
def get_prompt(prompt_type: str, language: str = "de") -> str:
    prompts = {
        "reduce_pdf": {
            "de": """Du erhältst den extrahierten Rohtext einer Stellenanzeige (PDF). Bitte filtere **nur** die relevanten Informationen heraus:

- Jobtitel
- Aufgabenbereiche
- Anforderungen (Fähigkeiten, Erfahrung, Ausbildung)
- Erwünschte Qualifikationen

Ignoriere irrelevante Inhalte wie:
- Navigationsmenüs der Website
- Kontaktinformationen
- Rechtliche Hinweise
- Hinweise zum Bewerbungsprozess

Gib die Informationen als strukturierte Markdown-Liste mit klaren Überschriften zurück (z. B. **Jobtitel**, **Aufgaben**, ...).

Input:
\"\"\"
{pdf_text}
\"\"\"""",

            "en": """You will receive the raw extracted text of a job posting (PDF). Please filter out **only** the relevant information:

- Job title
- Responsibilities
- Requirements (skills, experience, education)
- Preferred qualifications

Ignore irrelevant content such as:
- Website navigation elements
- Contact information
- Legal notices
- Application instructions

Return the information as a structured markdown list with clear headings (e.g., **Job Title**, **Responsibilities**, ...).

Input:
\"\"\"
{pdf_text}
\"\"\""""
        },
        "select_experiences": {
            "de": """Du bist Expert:in für Karriereberatung. Basierend auf der folgenden Stellenanzeige und einer Liste bisheriger Berufserfahrungen sollst du die **drei relevantesten Erfahrungen** auswählen, die am besten zu den Anforderungen passen.

### Stellenanzeige:
\"\"\"
{bereinigter_jobtext}
\"\"\"

### Berufserfahrungen (Tabellenformat):
\"\"\"
{csv_als_text}
\"\"\"

Bitte gib für jede der drei gewählten Erfahrungen einen kurzen Abschnitt zurück mit:

- Jobtitel
- Aufgaben/Rolle
- Zeitraum (falls vorhanden)
- Warum diese Erfahrung besonders relevant für die Stelle ist

Formuliere die Antwort so, dass sie direkt in einer Bewerbung verwendet werden kann.""",

            "en": """You are an expert in career consulting. Based on the following job description and a list of past work experiences, please identify the **three most relevant experiences** that best match the job requirements.

### Job Description:
\"\"\"
{bereinigter_jobtext}
\"\"\"

### Work Experiences (Table Format):
\"\"\"
{csv_als_text}
\"\"\"

Please return a short paragraph for each of the three selected experiences including:

- Job title
- Tasks/Role
- Duration (if available)
- Why this experience is especially relevant to the job

Write the response so it can be directly used in an application."""
        }
    }

    try:
        return prompts[prompt_type][language]
    except KeyError:
        raise ValueError(f"❌ Prompt oder Sprache nicht gefunden: '{prompt_type}' in '{language}'")

In [7]:
def is_prompt_within_token_limit(prompt: str, model: str = "gpt-4", max_tokens: int = 8192) -> bool:
    """
    Prüft, ob der gegebene Prompt unterhalb des Tokenlimits liegt.
    
    Args:
        prompt (str): Der gesamte Prompt (z. B. inkl. PDF-Text, CSV-Text usw.)
        model (str): Modellname ("gpt-4", "gpt-3.5-turbo", etc.)
        max_tokens (int): Maximale erlaubte Anzahl an Tokens (inkl. Antwort)

    Returns:
        bool: True, wenn innerhalb des Limits, sonst False
    """
    encoding = tiktoken.encoding_for_model(model)
    num_tokens = len(encoding.encode(prompt))
    print(f"📏 Prompt enthält {num_tokens} Tokens (Limit: {max_tokens})")
    return num_tokens < max_tokens

In [8]:
def count_tokens(text: str, model: str = "gpt-4") -> int:
    encoding = tiktoken.encoding_for_model(model)
    tokens = encoding.encode(text)
    token_count = len(tokens)
    return token_count

In [9]:
def validate_prompt_length(prompt: str, model: str = "gpt-4", max_tokens: int = 8192) -> None:
    """
    Bricht das Programm ab, wenn der Prompt zu viele Tokens enthält.
    """
    encoding = tiktoken.encoding_for_model(model)
    token_count = len(encoding.encode(prompt))
    
    if token_count > max_tokens:
        raise ValueError(f"❌ Prompt ist zu lang ({token_count} Tokens). Maximal erlaubt: {max_tokens}.\nBitte kürzen oder aufteilen.")

In [10]:
def split_rows_with_header_into_token_batches(df: pd.DataFrame, max_tokens_per_batch: int, model: str = "gpt-4") -> list[str]:
    """
    Zerlegt den DataFrame in Pakete aus Zeilen (jede Zeile = Eintrag),
    wobei jeder Block den Header (Spaltennamen) enthält.
    Gibt eine Liste von Textblöcken zurück, die unter dem Tokenlimit bleiben.
    """
    encoding = tiktoken.encoding_for_model(model)
    batches = []
    current_rows = []
    current_token_count = 0

    # Header vorbereiten
    header_text = "\n".join([f"{col}:" for col in df.columns]) + "\n\n"
    header_token_count = len(encoding.encode(header_text))

    if header_token_count >= max_tokens_per_batch:
        raise ValueError("❌ Der Header allein überschreitet das Tokenlimit.")

    for _, row in df.iterrows():
        # Zeile als formatierten Text
        row_text = "\n".join([f"{col}: {val}" for col, val in row.items() if pd.notna(val)]) + "\n\n"
        row_token_count = len(encoding.encode(row_text))

        if row_token_count >= max_tokens_per_batch - header_token_count:
            raise ValueError("❌ Einzelne Zeile + Header überschreiten das Tokenlimit.")

        if current_token_count + row_token_count + header_token_count > max_tokens_per_batch:
            # Block abschließen
            batch_text = header_text + "".join(current_rows)
            batches.append(batch_text.strip())
            # neuen Block beginnen
            current_rows = [row_text]
            current_token_count = row_token_count
        else:
            current_rows.append(row_text)
            current_token_count += row_token_count

    # letzten Block hinzufügen
    if current_rows:
        batch_text = header_text + "".join(current_rows)
        batches.append(batch_text.strip())

    return batches

In [22]:
def ask_chatgpt_single_prompt(
    prompt: str,
    lan: str = 'de',
    model: str = "gpt-4",
    temperature: float = 0.2,
    max_retries: int = 2,
    delay_seconds: float = 2.0,
) -> str:
    """
    Sendet einen einzelnen Prompt an die OpenAI-API (ab openai>=1.0.0) und gibt die Antwort als String zurück.
    """
    client = OpenAI()  # Liest den API-Key automatisch aus der Umgebungsvariable OPENAI_API_KEY
    if lan == 'de':
        system_instruction = "Du bist ein hilfreicher Assistent. Antworte bitte auf Deutsch."
    elif lan == 'en':
        system_instruction = "You are a helpful assistant. Please answer in English."

    for attempt in range(1, max_retries + 1):
        try:
            print("📤 Sende Anfrage an ChatGPT…")

            response = client.chat.completions.create(
                model=model,
                messages=[
                    {"role": "system", "content": system_instruction},
                    {"role": "user", "content": prompt}
                ],
                temperature=temperature
            )

            content = response.choices[0].message.content.strip()
            print("✅ Antwort erhalten.")
            return content

        except Exception as e:
            print(f"⚠️ Fehler bei der Anfrage (Versuch {attempt}): {e}")
            if attempt < max_retries:
                time.sleep(delay_seconds)
            else:
                return f"❌ Fehler nach {max_retries} Versuchen: {e}"

In [31]:
def create_profil(pdf_text, df, language='de'):
    
    prompt_text_job = get_prompt("reduce_pdf", language="de")
    filled_prompt_job = prompt_text_job.replace("{pdf_text}", pdf_text)
    
    validate_prompt_length(filled_prompt_job, model="gpt-4", max_tokens=8192)

    short_promt_job = ask_chatgpt_single_prompt(filled_prompt_job)

    prompt_text_df = get_prompt("select_experiences", language="de")
    filled_prompt_df = prompt_text_df.replace("{pdf_text}", short_promt_job)
    
    max_tokens = 8192
    limit_df = max_tokens - count_tokens(filled_prompt_df)

    batches = split_rows_with_header_into_token_batches(df, limit_df)

    while len(batches) > 1:
        profils = []
        for batch in batches:
            filled_prompt_df = filled_prompt_df.replace("{csv_als_text}", batch)
            validate_prompt_length(filled_prompt_df, model="gpt-4", max_tokens=8192)
            profil = ask_chatgpt_single_prompt(filled_prompt_job)
            profils.append(profil)
        batches = split_rows_with_header_into_token_batches(profils, limit_df) # problem beheben 
    if len(batches) == 1: 
        filled_prompt_df = filled_prompt_df.replace("{csv_als_text}", batches[0])
    output = 'output'
    
    return output

In [32]:
# filtern des dataframe
create_profil(pdf_text, df)

📤 Sende Anfrage an ChatGPT…
⚠️ Fehler bei der Anfrage (Versuch 1): Error code: 401 - {'error': {'message': 'Incorrect API key provided: your-ope************here. You can find your API key at https://platform.openai.com/account/api-keys.', 'type': 'invalid_request_error', 'param': None, 'code': 'invalid_api_key'}}
📤 Sende Anfrage an ChatGPT…
⚠️ Fehler bei der Anfrage (Versuch 2): Error code: 401 - {'error': {'message': 'Incorrect API key provided: your-ope************here. You can find your API key at https://platform.openai.com/account/api-keys.', 'type': 'invalid_request_error', 'param': None, 'code': 'invalid_api_key'}}
📤 Sende Anfrage an ChatGPT…
⚠️ Fehler bei der Anfrage (Versuch 1): Error code: 401 - {'error': {'message': 'Incorrect API key provided: your-ope************here. You can find your API key at https://platform.openai.com/account/api-keys.', 'type': 'invalid_request_error', 'param': None, 'code': 'invalid_api_key'}}
📤 Sende Anfrage an ChatGPT…
⚠️ Fehler bei der Anfrage (

AttributeError: 'list' object has no attribute 'columns'